In [32]:
from mesa import Agent, Model
from mesa.time import RandomActivation
import random

In [33]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
import random

# Customer Agent Class
class CustomerAgent(Agent):
    def __init__(self, unique_id, model, revenue, potential_revenue, relationship_strength, segment, product_quality, support_experience):
        super().__init__(unique_id, model)
        self.revenue = revenue
        self.potential_revenue = potential_revenue
        self.relationship_strength = relationship_strength
        self.segment = segment
        self.product_quality = product_quality
        self.support_experience = support_experience
        self.priority_score = 0
        self.nps_score = 0
        self.previous_nps = 0

    def prioritize(self):
        # Calculate priority score based on revenue potential and relationship strength
        self.priority_score = 0.6 * self.potential_revenue + 0.4 * self.relationship_strength

    def generate_nps(self):
        # Base satisfaction influenced by segment
        base_satisfaction = random.uniform(7, 10) if self.segment == 'Premium' else random.uniform(5, 8)

        # Adjust based on product quality
        product_effect = (self.product_quality - 5) / 10  # Normalize to a small range [-0.5, 0.5]

        # Adjust based on support experience
        support_effect = (self.support_experience - 5) / 10  # Normalize to a small range [-0.5, 0.5]

        # Incorporate historical NPS influence (e.g., trend over time)
        trend_effect = (self.previous_nps - 50) / 100  # Influence by previous score

        # Final satisfaction score
        satisfaction = base_satisfaction + product_effect + support_effect + trend_effect

        # Ensure satisfaction is within bounds
        satisfaction = max(1, min(10, satisfaction))

        # Categorize into NPS
        if satisfaction >= 9:
            self.nps_score = 100  # Promoter
        elif satisfaction >= 7:
            self.nps_score = 0    # Passive
        else:
            self.nps_score = -100 # Detractor

        # Store the current NPS for future reference
        self.previous_nps = self.nps_score

        return self.nps_score

    def feedback_loop(self):
        # Act on feedback
        if self.nps_score < 0:
            print(f"Customer {self.unique_id}: Detractor detected! Taking action.")
        else:
            print(f"Customer {self.unique_id}: Feedback noted.")

    def step(self):
        # Run the customer prioritization, NPS generation, and feedback loop
        self.prioritize()
        print(f"Customer {self.unique_id} prioritized with score {self.priority_score}.")

        nps_score = self.generate_nps()
        print(f"Customer {self.unique_id} NPS score: {nps_score}.")

        self.feedback_loop()

In [34]:
# Customer Experience Model Class
class CustomerExperienceModel(Model):
    def __init__(self, N):
        self.num_agents = N
        self.schedule = RandomActivation(self)

        # Create customers
        for i in range(self.num_agents):
            revenue = random.randint(5000, 20000)
            potential_revenue = random.randint(3000, 15000)
            relationship_strength = random.randint(1, 10)
            segment = 'Premium' if random.random() > 0.5 else 'Standard'
            product_quality = random.randint(5, 10)
            support_experience = random.randint(5, 10)

            agent = CustomerAgent(i, self, revenue, potential_revenue, relationship_strength, segment, product_quality, support_experience)
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()

In [35]:
def customer_portrayal(agent):
    if agent is None:
        return

    portrayal = {
        "Shape": "circle",
        "Filled": "true",
        "r": 0.5,
        "Layer": 0,
        "Color": "red" if agent.nps_score < 0 else "green",
        "text": agent.nps_score,
        "text_color": "white",
    }
    return portrayal

In [36]:
grid = CanvasGrid(customer_portrayal, 10, 10, 500, 500)
chart = ChartModule([{"Label": "NPS Score", "Color": "Black"}])

In [37]:
server = ModularServer(
    CustomerExperienceModel,
    [grid, chart],
    "Customer Experience Model",
    {"N": 10}
)

if __name__ == "__main__":
    server.port = 8521
    server.launch()

Interface starting at http://127.0.0.1:8521


C:\Users\user\AppData\Local\Temp\ipykernel_10032\795299476.py:10: FutureWarning: The Mesa Model class was not initialized. In the future, you need to explicitly initialize the Model by calling super().__init__() on initialization.
  super().__init__(unique_id, model)


OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted

NameError: name 'agents' is not defined